# Similitud Notebook
#### En este notebook podremos analizar con más facilidad la similitud entre las respuestas calificadas con mayor calificación con las de menor

In [ ]:
# importo librerias
import spacy
import numpy as np
import pandas as pd
from hunspell import Hunspell

h = Hunspell('es_MX', hunspell_data_dir = 'C:/Python/Lib/site-packages/dictionaries')
spa_lex = spacy.load('es_core_news_md')

In [ ]:
# con esto evito que se impriman 'errores'
import warnings
warnings.filterwarnings("ignore")

#### Importamos el CSV original como un dataframe

In [ ]:
# df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/Bases de datos/Secundaria/SECUNDARIA_TODO.csv")

df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/cuenta_de_palabras/WORDCOUNT_SECUNDARIA.csv")

#txt_file = open("C:/Users/Drablaguna/Desktop/ResFile.txt", "a+")
#txt_file.close()

#### Funciones

In [ ]:
# funciones para eliminar caracteres de puntuacion
def evalChar(x):
    if x in [",",".",":",";","`","'",'"',"(",")","-","_","~","/","?","¿","=","[","]","\n","\r"]:
        x = ""
    return x
def cleanString(x):
    s = map(evalChar, x)
    s = "".join(list(s))
    return s

In [ ]:
# funcion para inicializar el diccionario de resultados, para tener resultados diferentes por prueba
def start_dict():
    dic = {
    'token_base': [],      # tokens originales
    'token_toCompare': [], # token con el que se comparo
    'similarity': [],      # valor de la similitud de ambos
    'total_sim_mean': 0    # promedio similitud por frase
    }
    return dic

In [ ]:
# funcion para analizar
def analize(base, comparer):
    dic = start_dict()
    for token_base in base:
        for token_toCompare in comparer:
            simil = token_base.similarity(token_toCompare)
            dic['token_base'].append(token_base.text)
            dic['token_toCompare'].append(token_toCompare.text)
            dic['similarity'].append(simil)
    # genero el promedio de similitud que fue extraido de la sim de cada token con token
    dic['total_sim_mean'] = np.mean(dic['similarity'])
    return dic

In [ ]:
# funcion para analizar rapido y mostrar similitud
def quickize(master_row, slave_row):
#     master_row = checkSentence(master_row,h)
#     slave_row = checkSentence(slave_row,h)
    master_row = spa_lex(master_row)
    slave_row = spa_lex(slave_row)
    dic = analize(master_row, slave_row)
    return dic

# funcion para imprimir las palabras que se compararon y sus similitudes
def print_zip(dic):
    print(list(zip(dic['token_base'], dic['token_toCompare'], dic['similarity'])))

In [ ]:
# funcion para elegir la palabra correcta entre sus sugerencias
def checkWord(word, hun_dic, original_sentence):
    if not hun_dic.spell(word): # si la palabra NO se encuentra en el diccionario hunspell
        n = 0
        a = hun_dic.suggest(word)
        print("=> " + original_sentence)
        for x in a:
            print(str(n) + ": " + x)
            n+=1
        # se puede ingresar la respuesta correcta con el id o escribiendola manualmente
        correct_id_word = input("("+word+") ID respuesta correcta o ingresa la palabra: ")
        if correct_id_word.isdigit():
            correct = a[int(correct_id_word)]
        else:
            correct = correct_id_word
    else: # si la palabra se encuentra en el diccionario hunspell
        correct = word
    return correct.strip().lower()

# funcion para checar una oracion usando la funcion de escoger la palabra correcta
def checkSentence(s_string, hun_dic):
    new_sentence = []
    for token in s_string.split(" "): # se evaluara cada token si es una palabra existente o no
        correct_word = checkWord(token, hun_dic, s_string)
        if correct_word != "": # se agregara a la oracion nueva solo si es algo
            new_sentence.append(correct_word)
    #print("\n")
    return " ".join(new_sentence)
        
# s = "porque sii mas que nada"
# s = checkSentence(s,h)
# print(s)

# Resultado (SECU_TODO_CLEAN)

In [ ]:
# promedio similitud entre respuestas
#print('\n')
#print(list(zip(results['token_base'], results['token_toCompare'], results['similarity'])))

# Resultado (WORDCOUNT_SECUNDARIA)

In [ ]:
# aqui se han retirado las STOPWORDS
# de acuerdo al siguiente criterio de quickPlotter.py usando Spacy
# if token.is_stop or token.is_punct or token.is_quote or len(token) == 1
#print('\n')
#print(list(zip(results['token_base'], results['token_toCompare'], results['similarity'])))

### Dataframe base

In [ ]:
# WORDCOUNT SECUNDARIA ===========================================================
ans_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos_clean_ans"
eval_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos_eval"

# SECU TODO CLEAN ================================================================
# ans_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos"
# eval_col = "a5"

# obtengo los valores de respuestas limpias y sus evaluaciones en un dataframe
df_base = df[[ans_col, eval_col]]

### Dataframes evaluados a  3, 2, 1 y 0

In [ ]:
# df head testing

#df_three.head(60)
#df_three.shape

#df_two.head(3)
#df_two.shape

#df_one.head(3)
#df_one.shape

#df_zero.head(3)
#df_zero.shape

print('3 => '+str(len(df_three)))
print('2 => '+str(len(df_two)))
print('1 => '+str(len(df_one)))
print('0 => '+str(len(df_zero)))

In [ ]:
# funcion para imprimir rapido resultados de similitudes en una columna
"""
def print_df_res(df_master, df_slave, ans_col):
    counter = 1
    total_mean = []
    for row in df_master[ans_col]:
        master_row = cleanString(row) # la row limpia sin [] ni comas
        print("============================== "+master_row.upper()+" ==============================")
        for line in df_slave[ans_col]:
            slave_row = cleanString(line)
            if slave_row != "": # para evitar evaluaciones donde la fila este vacia
                r = quickize(master_row, slave_row)
                total_mean.append(r['total_sim_mean'])
                print(str(counter)+': '+str(r['total_sim_mean']))
                counter+=1
    print(np.mean(total_mean))
""" 

# funcion para evaluar 1 respuesta de 3 con todo
def print_df_res(master_row, df_slave, ans_col):
    counter = 1
    total_mean = []
    master_row = cleanString(master_row) # la row limpia sin [] ni comas
    #print("============================== "+master_row.upper()+" ==============================")
    for line in df_slave[ans_col]:
        slave_row = cleanString(line)
        if slave_row != "": # para evitar evaluaciones donde la fila este vacia
            r = quickize(master_row, slave_row)
            total_mean.append(r['total_sim_mean'])
            #print(str(counter)+': '+str(r['total_sim_mean']))
            print(str(r['total_sim_mean']))
            counter+=1
    print(f"\n{np.mean(total_mean)}")

def quickPrint(df, ans_col, eval_col, n1, n2):
    df_base  = df[[ans_col, eval_col]]
    
    df_three = df_base.loc[df_base[eval_col] == 3]
    df_two   = df_base.loc[df_base[eval_col] == 2]
    df_one   = df_base.loc[df_base[eval_col] == 1]
    df_zero  = df_base.loc[df_base[eval_col] == 0]
    
    #three_ans = "por que hace mucho ruido"
    three_ans = "porque grita mucho"
    
    # 3 con 3
    if n1 == 3 and n2 == 3:
        print_df_res(three_ans, df_three, ans_col)
        
    # 3 con 2
    if n1 == 3 and n2 == 2:
        print_df_res(three_ans, df_two, ans_col)
        
    # 3 con 1
    if n1 == 3 and n2 == 1:
        print_df_res(three_ans, df_one, ans_col)
        
    # 3 con 0
    if n1 == 3 and n2 == 0:
        print_df_res(three_ans, df_zero, ans_col)
    
    
    # 2 con 2
    if n1 == 2 and n2 == 2:
        print_df_res(three_ans, df_two, ans_col)
        
    # 2 con 1
    if n1 == 2 and n2 == 1:
        print_df_res(three_ans, df_one, ans_col)
        
    # 2 con 0
    if n1 == 2 and n2 == 0:
        print_df_res(three_ans, df_zero, ans_col)
        
    
    # 1 con 1
    if n1 == 1 and n2 == 1:
        print_df_res(three_ans, df_one, ans_col)
        
    # 1 con 0
    if n1 == 1 and n2 == 0:
        print_df_res(three_ans, df_zero, ans_col)
        
    
    # 0 con 0
    if n1 == 0 and n2 == 0:
        print_df_res(three_ans, df_zero, ans_col)

In [ ]:
quickPrint(df, ans_col, eval_col, 3,3)

In [ ]:
quickPrint(df, ans_col, eval_col, 3,2)

In [ ]:
quickPrint(df, ans_col, eval_col, 3,1)

In [ ]:
quickPrint(df, ans_col, eval_col, 3,0)

In [ ]:
quickPrint(df, ans_col, eval_col, 2,2)

In [ ]:
quickPrint(df, ans_col, eval_col, 2,1)

In [ ]:
quickPrint(df, ans_col, eval_col, 2,0)

In [ ]:
quickPrint(df, ans_col, eval_col, 1,1)

In [ ]:
quickPrint(df, ans_col, eval_col, 1,0)

In [ ]:
quickPrint(df, ans_col, eval_col, 0,0)